In [1]:
"""
Created on Friday 12th March 2021
@author: Aditya Asopa, Bhalla Lab, NCBS
"""
## Import libraries
import sys
import os
import importlib
import pathlib
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from eidynamics import ephys_classes
# from eidynamics.plot_maker import makeplots
from eidynamics.errors      import *


In [2]:
inputABFfile = "C:\\Users\\adity\\OneDrive\\NCBS\\Lab\\Projects\\EI_Dynamics\\Analysis\\testExamples\\testCells\\5211\\2021_09_07_0005_rec.abf"
cell_file= '' # presaved cell pickle file, optional

In [3]:
datafile      = pathlib.Path(inputABFfile)
exptDir       = datafile.parent
exptFile      = datafile.name
fileID        = exptFile[:15]
parameterFilePath = exptDir / str(fileID + "_experiment_parameters.py")
paramfileName = parameterFilePath.stem
parameterFilePath = parameterFilePath.parent

In [8]:
# Import Experiment Variables
try:
    print("Looking for experiment parameters locally")
    sys.path.append(str(parameterFilePath))
    exptParams = importlib.import_module(paramfileName, parameterFilePath)
    if not exptParams.datafile == exptFile:
        raise FileMismatchError()
    print('Experiment parameters loaded from: ', parameterFilePath)
except (FileMismatchError,FileNotFoundError) as err:
    print(err)
    print("No special instructions, using default variables.")
    import eidynamics.experiment_parameters_default as exptParams
    save_trial  = False
    print('Default Experiment Parameters loaded.\n'
            'Experiment will not be added to the cell pickle file,\n'
            'only excel file of cell response will be created.')
except Exception as err:
    print(err)
    print("Experiment Parameters error. Quitting!")
    sys.exit()

Looking for experiment parameters locally
Experiment parameters loaded from:  C:\Users\adity\OneDrive\NCBS\Lab\Projects\EI_Dynamics\Analysis\testExamples\testCells\5211


In [ ]:
 # Import stimulation coordinates
try:
    coordfileName   = exptParams.polygonProtocol
    if not coordfileName:
        raise FileNotFoundError
    coordfile       = os.path.join(os.getcwd(), "polygonProtocols", coordfileName)
    coordfile       = pathlib.Path.cwd() / "polygonProtocols" / coordfileName
    # os.path.isfile(coordfile)
    print('Local coord file loaded from: ', coordfile)
except FileNotFoundError:
    print('No coord file found, probably there isn\'t one')
    coordfile       = ''
except Exception as err:
    print(err)
    coordfile       = ''

In [ ]:
# loading or creating a cell object
try:
    print('Loading local cell data')
    cell = ephys_classes.Neuron.loadCell(cell_file)
except:
    print('Error in loading from file. Creating new cell.')
    cell = ephys_classes.Neuron(exptParams)

In [ ]:
cellFile = "C:\\Users\\adity\\OneDrive\\NCBS\\Lab\\Projects\\EI_Dynamics\\Analysis\\testExamples\\cell2941\\cell.pkl"

In [ ]:
cell.addExperiment(datafile=datafile, coordfile=coordfile, exptParams=exptParams)

In [ ]:
cell_file_pickle     = exptDir / str(str(exptParams.cellID) + "_temp.pkl")
cell_file_csv        = exptDir / str(str(exptParams.cellID) + "_temp.xlsx")

In [ ]:
ephys_classes.Neuron.saveCell(cell, cell_file_pickle)
cell.response.to_excel(cell_file_csv)